In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [ ]:
data_pd = pd.read_csv("C:\\Users\\VIRGIL\\anaconda34\\Ham10000\\HAM10000_metadata.csv")
data_pd.head()


,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [ ]:
train_dir = os.path.join('C:\\Users\\VIRGIL\\anaconda34\\Ham10000\\' ,'train_dir')
test_dir = os.path.join('C:\\Users\\VIRGIL\\anaconda34\\Ham10000\\', 'test_dir')


In [ ]:
df_count = data_pd.groupby('lesion_id').count()
df_count.head()


,image_id,dx,dx_type,age,sex,localization
lesion_id,,,,,,
HAM_0000001,1,1,1,1,1,1
HAM_0000002,3,3,3,3,3,3
HAM_0000005,4,4,4,4,4,4
HAM_0000006,3,3,3,3,3,3
HAM_0000009,3,3,3,3,3,3


In [ ]:
df_count = df_count[df_count['dx'] == 1]
df_count.reset_index(inplace=True)


     



In [ ]:
def duplicates(x):
    unique = set(df_count['lesion_id'])
    if x in unique:
        return 'no' 
    else:
        return 'duplicates'


     


In [ ]:

data_pd['is_duplicate'] = data_pd['lesion_id'].apply(duplicates)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,duplicates
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,duplicates
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,duplicates
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,duplicates
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,duplicates


In [ ]:
df_count = data_pd[data_pd['is_duplicate'] == 'no']


     





In [ ]:
train, test_df = train_test_split(df_count, test_size=0.1, stratify=df_count['dx'])

In [ ]:
def identify_trainOrtest(x):
    test_data = set(test_df['image_id'])
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'

#creating train_df
data_pd['train_test_split'] = data_pd['image_id'].apply(identify_trainOrtest)
train_df = data_pd[data_pd['train_test_split'] == 'train']
train_df.head()


,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate,train_test_split
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,duplicates,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,duplicates,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,duplicates,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,duplicates,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,duplicates,train


In [ ]:

test_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate
1206,HAM_0001841,ISIC_0025771,df,consensus,35.0,female,lower extremity,no
2753,HAM_0006465,ISIC_0030813,bcc,histo,65.0,male,back,no
2772,HAM_0006708,ISIC_0027189,bcc,histo,80.0,female,lower extremity,no
597,HAM_0000806,ISIC_0024371,bkl,histo,70.0,male,lower extremity,no
871,HAM_0003010,ISIC_0029924,bkl,consensus,55.0,female,face,no


In [ ]:
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])

In [ ]:
len(test_list)


    
len(train_list)


4496

In [ ]:
len(test_list)

119

In [ ]:

# Set the image_id as the index in data_pd
data_pd.set_index('image_id', inplace=True)

In [ ]:
os.mkdir(train_dir)
os.mkdir(test_dir)


In [ ]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [ ]:
for i in targetnames:
  directory1=train_dir+'/'+i
  directory2=test_dir+'/'+i
  os.mkdir(directory1)
  os.mkdir(directory2)



In [ ]:

for image in train_list:
    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join('C:\\Users\\VIRGIL\\anaconda34\\Ham10000\\All_Images', file_name)

    # copying the image from the source to target file
    target = os.path.join(train_dir, label, file_name)

    shutil.copyfile(source, target)


In [ ]:
for image in test_list:

    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join('C:\\Users\\VIRGIL\\anaconda34\\Ham10000\\All_Images', file_name)

    # copying the image from the source to target file
    target = os.path.join(test_dir, label, file_name)

    shutil.copyfile(source, target)

In [ ]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

# Augmenting images and storing them in temporary directories 
for img_class in targetnames:

    #creating temporary directories
    # creating a base directory
    Aug_dir = 'Aug_dir'
    os.mkdir(Aug_dir)
    # creating a subdirectory inside the base directory for images of the same class
    img_dir = os.path.join(Aug_dir, 'img_dir')
    os.mkdir(img_dir)

    img_list = os.listdir('C:\\Users\\VIRGIL\\anaconda34\\Ham10000\\train_dir\\' + img_class)

    # Copy images from the class train dir to the img_dir 
    for file_name in img_list:

        # path of source image in training directory
        source = os.path.join('C:\\Users\\VIRGIL\\anaconda34\\Ham10000\\train_dir\\' + img_class, file_name)

        # creating a target directory to send images 
        target = os.path.join(img_dir, file_name)

        # copying the image from the source to target file
        shutil.copyfile(source, target)

    # Temporary augumented dataset directory.
    source_path = Aug_dir

    # Augmented images will be saved to training directory
    save_path = 'C:\\Users\\VIRGIL\\anaconda34\\Ham10000\\train_dir\\' + img_class

    # Creating Image Data Generator to augment images
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(featurewise_center=False, 
                                    samplewise_center=False, 
                                    featurewise_std_normalization=False, 
                                    samplewise_std_normalization=False, 
                                    zca_whitening=False, 
                                    zca_epsilon=1e-06, 
                                    rotation_range=15, 
                                    width_shift_range=0.15, 
                                    height_shift_range=0.15, 
                                    brightness_range=None, 
                                    shear_range=0.1, 
                                    zoom_range=0.1, 
                                    channel_shift_range=0.1, 
                                    fill_mode='nearest', 
                                    cval=0.0, 
                                    horizontal_flip=True, 
                                    vertical_flip=True, 
                                    rescale=None, 
                                    preprocessing_function=None, 
                                    data_format='channels_last', 
                                    validation_split=0.0, 
                                    interpolation_order=1, 
                                    dtype='float32')

    batch_size = 50

    Aug_datagen = datagen.flow_from_directory(source_path,save_to_dir=save_path,save_format='jpg',target_size=(450, 600),batch_size=batch_size)

    # Generate the augmented images
    Aug_images = 1450

    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((Aug_images - num_files) / batch_size))

    # creating 8000 augmented images per class
    for i in range(0, num_batches):
        images, labels = next(Aug_datagen)

    # delete temporary directory 
    shutil.rmtree('Aug_dir')

Found 312 images belonging to 1 classes.
Found 496 images belonging to 1 classes.
Found 1054 images belonging to 1 classes.
Found 111 images belonging to 1 classes.
Found 1090 images belonging to 1 classes.
Found 1297 images belonging to 1 classes.
Found 136 images belonging to 1 classes.
